In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

# File to Load (Remember to Change These)
schools_path = "Resources/schools_complete.csv"
students_path = "Resources/students_complete.csv"

# read files
schools_df = pd.read_csv(schools_path)
students_df = pd.read_csv(students_path)

# merge
merge_df = students_df.merge(schools_df, how = 'left', on = 'school_name')
merge_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [72]:
# DISTRICT SUMMARY

# Create a high level snapshot (in table form) of the district's key metrics, including:
# Total Schools
# Total Students
# Total Budget
# Average Math Score
# Average Reading Score
# % Passing Math
# % Passing Reading
# Overall Passing Rate (Average of the above two)
# -------------1st Try---------
# district_summary = pd.DataFrame()
# district_summary['Total Schools'] = [data1['school_name'].nunique()]
# district_summary['Total Students'] = data2['Student ID'].nunique()
# district_summary['Total Budget'] = data1['budget'].sum()
# district_summary['Average Math Score'] = round(data2['math_score'].mean(),2)
# district_summary['Average Reading Score'] = round(data2['reading_score'].mean(),2)
# district_summary['% Passing Math'] = round(data2.loc[data2['math_score'] >= 60, 'math_score'].count()/ district_summary['Total Students'] * 100,2)
# district_summary['% Passing Reading'] = round(data2.loc[data2['reading_score'] >= 60, 'reading_score'].count() / district_summary['Total Students'] * 100,2)
# district_summary['% Overall Passing Rate'] = round(np.mean([district_summary['% Passing Math'],district_summary['% Passing Reading']]),2)
# district_summary

# -------------2nd try----------
school_count = merge_df['school_name'].nunique()
total_students = merge_df['student_name'].count()
total_budget = schools_df['budget'].sum()
avg_math = students_df['math_score'].mean()
avg_read = students_df['reading_score'].mean()
pass_math = students_df.loc[students_df['math_score'] > 59]['math_score'].count()
pct_pass_math = pass_math / total_students
pass_read = students_df.loc[students_df['reading_score'] > 59]['reading_score'].count()
pct_pass_read = pass_read / total_students
pass_rate_all = (pct_pass_math + pct_pass_read) / 2

district_summary = pd.DataFrame ({
    "Total Schools":[school_count],
    "Total Students":[total_students],
    "Total Budget":[total_budget],
    "Average Math Score":[avg_math],
    "Average Reading Score":[avg_read],
    "% Passing Math":[pct_pass_math],
    "% Passing Reading":[pct_pass_read],
    "Overall Passing Rate":[pass_rate_all],
})

district_summary.style.format({
    "Total Students":"{:,}",
    "Total Budget":"${:,.2f}",
    "Average Math Score":"{:,.2f}",
    "Average Reading Score":"{:,.2f}",
    "% Passing Math":"{:.1%}",
    "% Passing Reading":"{:.1%}",
    "Overall Passing Rate":"{:.1%}"
})

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,92.4%,100.0%,96.2%


In [87]:
# SCHOOLS SUMMARY

# Create an overview table that summarizes key metrics about each school, including:
# School Name
# School Type
# Total Students
# Total School Budget
# Per Student Budget
# Average Math Score
# Average Reading Score
# % Passing Math
# % Passing Reading
# Overall Passing Rate (Average of the above two)

# -------------1st Try---------
#grade_bins = np.array([0,59.99,100])
#labels = [0,1]
#data2['math_pass'] = pd.cut(data2['math_score'], bins = grade_bins, labels = labels)
#data2['reading_pass'] = pd.cut(data2['reading_score'], bins = grade_bins, labels = labels)
#data2['math_pass'] = pd.to_numeric(data2['math_pass'])
#data2['reading_pass'] = pd.to_numeric(data2['reading_pass'])

#school_type = data2.groupby(['school_name'])
#total_student = data_all['Student ID'].nunique()
#school_budget = data_all.set_index('school_name')['budget']
#per_student_budget = school_budget / data_all.set_index('school_name')['size']
#math_avg = data_all['math_score'].mean()
#read_avg = data_all['reading_score'].mean()
#pct_pass_math = data_all['math_pass'].sum() / total_student * 100
#pct_pass_read = data_all['reading_pass'].sum() / total_student * 100
#pct_pass_all = (pct_pass_math + pct_pass_read) / 2

#all_school_df = pd.DataFrame({
#    "School Type": school_type,
#    "Total Students": total_student,
#    "Total School Budget": school_budget,
#    "Per Student Budget": per_student_budget,
#    "Average Math Score": avg_math_score,
#    "Average Reading Score": avg_reading_score,
#    "% Passing Math": pct_pass_math,
#    "% Passing Reading": pct_pass_read,
#    "Overal Passing Rate": pct_pass_all
#})
# -------------2nd try----------

# Group by schools
grp_school = merge_df.set_index('school_name').groupby(['school_name'])

#school types
school_types = schools_df.set_index('school_name')['type']

# total students
student_per_school = grp_school['Student ID'].count()

# school budget
sch_budget = schools_df.set_index('school_name')['budget']

# per student budget
student_budget = schools_df.set_index('school_name')['budget']/schools_df.set_index('school_name')['size']

#avg scores by school
avg_math = grp_school['math_score'].mean()
avg_read = grp_school['reading_score'].mean()

# % passing scores
pass_math = merge_df[merge_df['math_score'] >= 59].groupby('school_name')['Student ID'].count()/student_per_school
pass_read = merge_df[merge_df['reading_score'] >= 59].groupby('school_name')['Student ID'].count()/student_per_school 
overall = merge_df[(merge_df['reading_score'] >= 59) & (merge_df['math_score'] >= 59)].groupby('school_name')['Student ID'].count()/student_per_school

school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": student_per_school,
    "Per Student Budget": student_budget,
    "Total School Budget": sch_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
})


school_summary = school_summary[['School Type', 
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Average Math Score', 
                          'Average Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          'Overall Passing Rate']]

school_summary.style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,}", 
                          "Per Student Budget": "${:.2f}",
                          'Average Math Score': "{:.2f}", 
                          'Average Reading Score': "{:.2f}", 
                          "% Passing Math": "{:.2%}", 
                          "% Passing Reading": "{:.2%}", 
                          "Overall Passing Rate": "{:.2%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928",$628.00,77.05,81.03,91.64%,100.00%,91.64%
Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.06,83.98,100.00%,100.00%,100.00%
Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71,81.16,90.71%,100.00%,90.71%
Ford High School,District,"2,739","$1,763,916",$644.00,77.10,80.75,91.75%,100.00%,91.75%
Griffin High School,Charter,"1,468","$917,500",$625.00,83.35,83.82,100.00%,100.00%,100.00%
Hernandez High School,District,"4,635","$3,022,020",$652.00,77.29,80.93,90.94%,100.00%,90.94%
Holden High School,Charter,427,"$248,087",$581.00,83.80,83.81,100.00%,100.00%,100.00%
Huang High School,District,"2,917","$1,910,635",$655.00,76.63,81.18,91.02%,100.00%,91.02%
Johnson High School,District,"4,761","$3,094,650",$650.00,77.07,80.97,91.33%,100.00%,91.33%
Pena High School,Charter,962,"$585,858",$609.00,83.84,84.04,100.00%,100.00%,100.00%


In [95]:
# Top Performing Schools (By Passing Rate)

# Create a table that highlights the top 5 performing schools based on Overall Passing Rate. Include:
#  * School Name
#  * School Type
#  * Total Students
#  * Total School Budget
#  * Per Student Budget
#  * Average Math Score
#  * Average Reading Score
#  * % Passing Math
#  * % Passing Reading
#  * Overall Passing Rate (Average of the above two)

top_5 = school_summary.sort_values("Overall Passing Rate" , ascending = False)
top_5.head().style.format({
    "Total Students": '{:,}',
    "Total School Budget": "${:,}",
    "Average Math Score": "{:.2f}",
    "Average Reading Score": "{:.2f}",
    "% Passing Math": "{:.1%}",
    "% Passing Reading": "{:.1%}",
    "Overall Passing Rate": "{:.1%}"
})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356",582,83.06,83.98,100.0%,100.0%,100.0%
Griffin High School,Charter,"1,468","$917,500",625,83.35,83.82,100.0%,100.0%,100.0%
Holden High School,Charter,427,"$248,087",581,83.80,83.81,100.0%,100.0%,100.0%
Pena High School,Charter,962,"$585,858",609,83.84,84.04,100.0%,100.0%,100.0%
Shelton High School,Charter,"1,761","$1,056,600",600,83.36,83.73,100.0%,100.0%,100.0%


In [96]:
# Bottom Performing Schools (By Passing Rate)

# * Create a table that highlights the bottom 5 performing schools based on Overall Passing Rate. Include all of the same metrics as above.

bottom_5 = school_summary.sort_values("Overall Passing Rate" , ascending = True)
bottom_5.head().style.format({
    "Total Students": '{:,}',
    "Total School Budget": "${:,}",
    "Average Math Score": "{:.2f}",
    "Average Reading Score": "{:.2f}",
    "% Passing Math": "{:.1%}",
    "% Passing Reading": "{:.1%}",
    "Overall Passing Rate": "{:.1%}"
})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Figueroa High School,District,"2,949","$1,884,411",639,76.71,81.16,90.7%,100.0%,90.7%
Rodriguez High School,District,"3,999","$2,547,363",637,76.84,80.74,90.9%,100.0%,90.9%
Hernandez High School,District,"4,635","$3,022,020",652,77.29,80.93,90.9%,100.0%,90.9%
Huang High School,District,"2,917","$1,910,635",655,76.63,81.18,91.0%,100.0%,91.0%
Johnson High School,District,"4,761","$3,094,650",650,77.07,80.97,91.3%,100.0%,91.3%


In [106]:
# Math Scores by Grade\*\*

# * Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
ninth_math = students_df.loc[students_df['grade'] == '9th'].groupby('school_name')["math_score"].mean()
tenth_math = students_df.loc[students_df['grade'] == '10th'].groupby('school_name')["math_score"].mean()
eleventh_math = students_df.loc[students_df['grade'] == '11th'].groupby('school_name')["math_score"].mean()
twelfth_math = students_df.loc[students_df['grade'] == '12th'].groupby('school_name')["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math
})

math_scores = math_scores[[
    '9th', 
    '10th', 
    '11th', 
    '12th'
]]

math_scores.style.format({
    '9th': "{:.2f}",      
    "10th": "{:.2f}",    
    "11th": "{:.2f}",     
    "12th": "{:.2f}"
})

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [107]:
# Reading Scores by Grade

# * Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
ninth_math = students_df.loc[students_df['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
tenth_math = students_df.loc[students_df['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
eleventh_math = students_df.loc[students_df['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
twelfth_math = students_df.loc[students_df['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

read_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math
})

read_scores = read_scores[[
    '9th', 
    '10th', 
    '11th', 
    '12th'
]]

read_scores.style.format({
    '9th': "{:.2f}",      
    "10th": "{:.2f}",    
    "11th": "{:.2f}",     
    "12th": "{:.2f}"
})

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
